In [ ]:
#CODE TO SETUP WANDB
!pip install wandb
 
import wandb
from wandb.keras import WandbCallback

wandb.login()

In [2]:
%matplotlib inline
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sys
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization,SpatialDropout1D,Bidirectional, Embedding, LSTM
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.utils import class_weight, compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import re
from tensorflow.keras.optimizers import Adam

np.set_printoptions(threshold=sys.maxsize)

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)


The dataset is structure:
|      feature      | Description | 
|------------|-----|
| engine_ID | The ID of a particular engine. |
| cycle | cycle count of a particular engine. |
| setting1, setting2, setting3 | different operating modes. |
| sensor1 -> sensor 21 | measurements of the 21 different sensors. |
| ttf = time to failure | how many more cycles the motor can run before a failure occurs. |

In [ ]:
# Loading the dataset
dataset = pd.read_csv('failures.csv')
dataset.head()

,engine_id,cycle,setting1,setting2,setting3,temp_fan_inlet,temp_lpc_outlet,temp_hpc_outlet,temp_lpt_outlet,pressure_fan_inlet,pressure_bypass_duct,pressure_hpc_outlet,physical_fan_speed,physical_core_speed,engine_pressure_ratio,static_pressure_hpc_outlet,fuel_flow_ration_ps30,corrected_fan_speed,corrected_core_speed,bypass_ratio,burner_fuel_air_ratio,bleed_enthalpy,demanded_fan_speed,demanded_corrected_fan_speed,hpt_collant_bleed,lpt_coolant_bleed,ttf
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


First approach is to solve this taks by doing classification

In [ ]:
# Engine is considered failing if "time to failure" is less or equal to 50
dataset['fail']=(dataset['ttf']<=50).astype(int)
dataset.drop(['ttf'],axis=1, inplace=True)

In [ ]:
# Split into training set and test set
# First 75 engines are used for training, the remaining for testing
train_engs=dataset['engine_id'].unique()[:75]
test_engs=dataset['engine_id'].unique()[75:]
dataset = dataset.set_index(['engine_id']).sort_index()
train=dataset.loc[train_engs]
test=dataset.loc[test_engs]
# train = train.reset_index()
# test = test.reset_index()

X_train = train.drop(['fail'], axis=1)
y_train = train['fail']

X_test = test.drop(['fail'], axis=1)
y_test = test['fail']

X_cols=X_train.columns

In [ ]:
# Scaling
scaler = preprocessing.MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=X_cols)
X_test = pd.DataFrame(X_test, columns=X_cols)

In [ ]:
# First using a simple Random Forest Trees model to determine if row is failure or not
RF_model = RandomForestClassifier(n_estimators=200, class_weight="balanced", verbose=1, n_jobs=-1)

RF_model.fit(X_train, y_train)

y_pred = RF_model.predict(X_test)

print(classification_report(y_test,y_pred))

print(accuracy_score(y_test,y_pred)*100)

print(confusion_matrix(y_test,y_pred))
#This turnsout pretty good, but we can do better

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s


              precision    recall  f1-score   support

           0       0.95      0.97      0.96      4197
           1       0.88      0.82      0.85      1275

    accuracy                           0.93      5472
   macro avg       0.91      0.89      0.90      5472
weighted avg       0.93      0.93      0.93      5472

93.23830409356725
[[4057  140]
 [ 230 1045]]


[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.1s finished


In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights)),
class_weights=class_weights[0]

In [ ]:
# Normal NN just to compare with the random forest model

batch_size = 64
epochs = 50
dropoutrate = 0.1
input_dim = X_train.shape[1]
activation_hidden = 'relu' 
activation_output = 'sigmoid'
initializer = 'RandomUniform' 
SGD = tf.keras.optimizers.SGD()
adam = tf.keras.optimizers.Adam(0.001)
RMSProp = tf.keras.optimizers.RMSprop()
Adagrad= tf.keras.optimizers.Adagrad()

model = Sequential()
model.add(Dense(128, input_dim=input_dim, kernel_initializer=initializer,activation=activation_hidden))
model.add(BatchNormalization())
model.add(Dropout(dropoutrate))
model.add(Dense(64, kernel_initializer=initializer,activation=activation_hidden))
model.add(BatchNormalization())
model.add(Dropout(dropoutrate))
model.add(Dense(1, kernel_initializer=initializer,activation=activation_output))


model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])

# train the model

history = model.fit(X_train, y_train, validation_split=0.3, class_weight=class_weights ,epochs=epochs, verbose=1)

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

print(classification_report(y_test,y_pred))

print(accuracy_score(y_test,y_pred)*100)

print(confusion_matrix(y_test,y_pred))

#The NN has a harder time correctly classifying the failures resulting in a lower precision and accuracy.

              precision    recall  f1-score   support

           0       0.98      0.87      0.92      4197
           1       0.68      0.94      0.79      1275

    accuracy                           0.88      5472
   macro avg       0.83      0.90      0.86      5472
weighted avg       0.91      0.88      0.89      5472

88.45029239766082
[[3639  558]
 [  74 1201]]


In [ ]:
dataset = pd.read_csv('failures.csv')

In [ ]:
# A better solution should be to see the data as a time series and use a recurrent neural network

#Failure is same as above
fail_period =50
dataset['failure'] = np.where(dataset['ttf']<=fail_period,1,0)

# Firs 75 engines are used for training, the remaining for testing
dataset_train = dataset[dataset['engine_id']<=75].copy()
dataset_test =  dataset[dataset['engine_id']>75].copy()

features_col_name=dataset.columns[1:-2]
MinMax = MinMaxScaler()
MinMax.fit(dataset_train[features_col_name])

dataset_train[features_col_name]=MinMax.transform(dataset_train[features_col_name])
dataset_test[features_col_name]= MinMax.transform(dataset_test[features_col_name])

seq_length = 50

# creating training set, a sliding window is created to feed into the model. This way it is possible to predict if the next row is a failure, instead of the current one

X_LSTM = []
y_LSTM = []

for e in range(dataset_train['engine_id'].min(),dataset_train['engine_id'].max()+1):
    selection = np.asarray(dataset_train.loc[dataset_train['engine_id'] == e])
    X = np.zeros((seq_length-1,28))
    X = np.vstack((X,selection))
    for c in range(0,X.shape[0]-seq_length+1):
        X_LSTM.append(X[c:c+seq_length,:])
        y_LSTM.append(selection[c,27])
X_train = np.asarray(X_LSTM)[:,:,2:26]
y_train = np.asarray(y_LSTM)

X_train,y_train = shuffle(X_train,y_train)


# compiling test set same way as training set

labels = dataset_test['failure'].values
X_LSTM = []
y_LSTM = []

for e in range(dataset_test['engine_id'].min(),dataset_test['engine_id'].max()+1):
    selection = np.asarray(dataset_test.loc[dataset_test['engine_id'] == e])
    X = np.zeros((seq_length-1,28))
    X = np.vstack((X,selection))
    for c in range(0,X.shape[0]-seq_length+1):
        X_LSTM.append(X[c:c+seq_length,:])
        y_LSTM.append(selection[c,27])
X_test = np.asarray(X_LSTM)[:,:,2:26]
y_test = np.asarray(y_LSTM)

X_test,y_test = shuffle(X_test,y_test)




In [ ]:
#Using class weights to balance the data
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights)),
class_weights=class_weights[0]

In [ ]:
# Creating the LSTM model, with use of bidirectional LSTM layers. This way the data is read from left to right and from right to left.
def train_model():
    run = wandb.init(project='maintenance',
                    config={  # and include hyperparameters and metadata
                        "learning_rate": 0.01,
                        "epochs": 10,
                        "batch_size": 64,
                        "dropoutrate":0.1,
                        "activation_hidden": 'relu' ,
                        "activation_output": 'sigmoid',
                        "loss_function": "binary_crossentropy",
                        "architecture": "LSTM",
                        "dataset": "Engine_failure"
                    })
    
    config = wandb.config
    adam = Adam(config.learning_rate)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    model = Sequential()
    #-----------------------------------------------
    model.add(Bidirectional(LSTM(units = 64,return_sequences=True ,input_shape=(X_train.shape[1], X_train.shape[2]))))
    model.add(BatchNormalization())
    model.add(Activation(config.activation_hidden))
    model.add(Dropout(config.dropoutrate))
    #-----------------------------------------------
    model.add(Bidirectional(LSTM(units = 32,return_sequences=False)))
    model.add(BatchNormalization())
    model.add(Activation(config.activation_hidden))
    model.add(Dropout(config.dropoutrate))
    #-----------------------------------------------
    model.add(Dense(16))
    model.add(BatchNormalization())
    model.add(Activation(config.activation_hidden))
    model.add(Dropout(config.dropoutrate))
    model.add(Dense(1, activation=config.activation_output))

    model.compile(loss=config.loss_function,
                optimizer=adam,
                metrics=['accuracy'])

    # Trainen
    history = model.fit(X_train,y_train,batch_size=config.batch_size, epochs=config.epochs,class_weight=class_weights ,validation_split=0.2, verbose=1, callbacks=[WandbCallback(),es])
    loss, accuracy = model.evaluate(X_test, y_test)
    wandb.log({'Test loss': loss, 'Test accuracy': accuracy})
    return model, history,run,checkpoint_filepath

In [ ]:
sweep_config = {
  'method': 'random', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'parameters': {
      'batch_size': {
          'values': [8, 16, 32, 64, 128, 256]
      },
      'learning_rate':{
          'min': 0.00001,
          'max': 0.1
      },
      'dropoutrate':{
          'values': [0.1, 0.2, 0.3,0.4]
      },
  },
  'early_terminate':{
      'type': 'hyperband',  
      'min_iter': 5
    }
}

sweep_id = wandb.sweep(sweep_config, project="maintenance'")
wandb.agent(sweep_id, function=train_model)

In [ ]:
model, history,run,checkpoint_filepath = train_model()
#Sweeps are used to find the optimal hyperparameters for the model.

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

print(classification_report(y_test,y_pred))

print(accuracy_score(y_test,y_pred)*100)

print(confusion_matrix(y_test,y_pred))
#The LSTM approach is the best performing, with RF very close. However this model is prefered as it is able to predict if a engine will fail one row in advance comapared to the RF model.

              precision    recall  f1-score   support

         0.0       0.98      0.93      0.95      4197
         1.0       0.80      0.94      0.86      1275

    accuracy                           0.93      5472
   macro avg       0.89      0.94      0.91      5472
weighted avg       0.94      0.93      0.93      5472

93.11038011695906
[[3891  306]
 [  71 1204]]
